In [ ]:
import re
import numpy as np
from scipy.interpolate import UnivariateSpline
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import seaborn as sns
import kw

In [ ]:
# データの読み込み

data = kw.getdf_xlsx()['sheet1']
data.head(2)

サンプルが小さい場合、サンプルを増やす

In [ ]:
# ランダムな日付の生成
import numpy as np
import random
from datetime import datetime, timedelta

# 開始日と終了日の設定
start_date_str = '20150101'  # 開始日（文字列形式）
end_date = datetime.now()  # 終了日（現在の日付）

# 文字列形式から日付形式に変換
start_date = datetime.strptime(start_date_str, '%Y%m%d')

# ランダムな日付を生成
n_dates = 100
random_dates = [start_date + (end_date - start_date) * random.random() for _ in range(n_dates)]
random_dates = sorted(random_dates)  # 日付をソートする

# 元のデータと同じ形式に日付を変換
random_dates = [date.strftime('%Y%m%d') for date in random_dates]

random_dates[:5]  # 最初の5つの日付を表示

In [ ]:
# Create a new dataframe with the new TESTDATEs and test_no
new_data = pd.DataFrame([(date, test_no) for date in random_dates for test_no in range(1, 7)], 
                        columns=['TESTDATE', 'test_no'])

# Show the first few rows of the new data
new_data.head(2)

In [ ]:
# Merge the new data with the original data
data = pd.concat([data, new_data], ignore_index=True)

# TESTDATEを日付形式に変換
data['TESTDATE'] = pd.to_datetime(data['TESTDATE'], format='%Y%m%d')

In [ ]:
# Create a lookup table from the data for TESTDATE = 2015-09-11
lookup_table = data[data['TESTDATE'] == '2015-09-11'].set_index('test_no')['capacity']

# Fill NaNs in the 'capacity' column using the lookup table
data.loc[data['capacity'].isna(), 'capacity'] = data['test_no'].map(lookup_table)

In [ ]:
# Set the seed for reproducibility
np.random.seed(0)

# List of columns to fill
columns_to_fill = ['total_head', 'current', 'output', 'shaft_power']

# Standard deviation of the noise
noise_std = 0.01

# List of unique TESTDATEs
test_dates = data['TESTDATE'].unique()

# Create the lookup table for '2015-09-11'
lookup_table_base = data[(data['TESTDATE'] == '2015-09-11')].set_index('test_no')

for column in columns_to_fill:
    for test_date in test_dates:
        # Add noise to the lookup table values
        lookup_table = lookup_table_base[column].copy()
        noise = np.random.normal(0, noise_std, size=len(lookup_table))
        lookup_table += noise

        # Fill NaNs in the current column using the lookup table
        data.loc[(data['TESTDATE'] == test_date) & data[column].isna(), column] = data.loc[(data['TESTDATE'] == test_date) & data[column].isna(), 'test_no'].map(lookup_table)

data.head()

しっかりしたデータの場合はここから

In [ ]:
data['theoretical_power'] = data['capacity'] * data['total_head'] * 9.8 / 60
data['pump_eff'] = data['theoretical_power'] / data['shaft_power'] * 100

In [ ]:
# Set the style and size of the plot
sns.set(style="whitegrid")
plt.figure(figsize=(8, 10))

# List of variables to plot against capacity
variables = ['total_head', 'current', 'output', 'shaft_power', 'pump_eff']

# Create a subplot for each variable
for i, variable in enumerate(variables):
    plt.subplot(len(variables), 1, i+1)
    
    # Create a line plot for each test date
    for test_date in data['TESTDATE'].unique():
        subset = data[data['TESTDATE'] == test_date]
        # Convert the test date to string for labeling
        # sns.lineplot(x='capacity', y=variable, data=subset, label=str(test_date))
        sns.lineplot(x='capacity', y=variable, data=subset)
        
    # Add a title to the subplot
    plt.title(f'Change in {variable} with respect to capacity')
    # plt.legend(loc='best', bbox_to_anchor=(1, 0.5))

# Adjust the layout
plt.tight_layout()
plt.show()



In [ ]:
# List of features
features = ['total_head', 'current', 'output', 'shaft_power', 'pump_eff']  # replace with your actual feature names

# For each feature
for feature in features:
    
    # Set up the figure and axes for the 4 plots
    fig, ax = plt.subplots(1, 4, figsize=(20, 5))

    # Histogram
    sns.histplot(data, x=feature, hue="test_no", multiple="stack", kde=True, ax=ax[0])
    ax[0].set_title(f'Histogram of {feature} by test_no')

    # Box plot
    sns.boxplot(x="test_no", y=feature, data=data, ax=ax[1])
    ax[1].set_title(f'Box plot of {feature} by test_no')

    # Scatter plot
    sns.scatterplot(x="capacity", y=feature, hue="test_no", data=data, ax=ax[2])
    ax[2].set_title(f'Scatter plot of {feature} vs capacity by test_no')

    # Time series plot
    sns.lineplot(x="TESTDATE", y=feature, hue="test_no", data=data.sort_values(['TESTDATE', 'test_no']), ax=ax[3])
    ax[3].set_title(f'Time series plot of {feature} by test_no')

    plt.tight_layout()
    plt.show()



In [ ]:
for feature in features:
    summary = data.groupby('test_no')[features].describe()
    print(summary)

In [ ]:
for i, feature in enumerate(features):
    summary = data.groupby('test_no')[feature].describe()
    # ファイルに書き出し
    summary.to_csv(f'summary_{i}.csv')

# それぞれのファイルを読み込んで表示
for i in range(len(features)):
    print(pd.read_csv(f'summary_{i}.csv'))


In [ ]:
data.describe()

In [ ]:
from scipy import stats
import statsmodels.api as sm

# For each feature, calculate skewness and kurtosis using scipy.stats
for feature in features:
    skewness = stats.skew(data[feature])
    kurtosis = stats.kurtosis(data[feature])
    print(f"Feature: {feature}, Skewness: {skewness:.2f}, Kurtosis: {kurtosis:.2f}")

# Perform regression analysis using statsmodels
X = data[['total_head', 'current', 'shaft_power']]
Y = data['output']
X = sm.add_constant(X)  # Add a constant column to the predictors

model = sm.OLS(Y, X)
results = model.fit()

# Print out the regression results
print(results.summary())


In [ ]:
data.groupby("test_no").describe()

In [ ]:
# ペアプロット
# Pairplot for each test_no
for test_no in data['test_no'].unique():
    sns.pairplot(data[data['test_no'] == test_no][features])
    plt.title(f'Pairplot for test_no {test_no}')
    plt.show()


In [ ]:
# 相関ヒートマップ
# Correlation heatmap for each test_no
for test_no in data['test_no'].unique():
    corr = data[data['test_no'] == test_no][features].corr()
    
    plt.figure(figsize=(5, 4))
    sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title(f'Correlation heatmap for test_no {test_no}')
    plt.show()

In [ ]:
# 統計的手法

from scipy.stats import zscore

# Calculate the z-score for each feature in each test_no
for test_no in data['test_no'].unique():
    data.loc[data['test_no'] == test_no, features] = data[data['test_no'] == test_no][features].apply(zscore)
    
# Mark outliers as True (absolute z-score greater than 3)
data['outlier'] = data[features].apply(lambda x: (abs(x) > 3).any(), axis=1)

# Display the first few rows of the updated data
data.head()


In [ ]:
from scipy.stats import zscore

# For each test_no
for test_no in data['test_no'].unique():

    # Select the data for this test_no
    data_test = data[data['test_no'] == test_no][features]
    outliers_test = data[data['test_no'] == test_no]['outlier']
    
    # Scale the data
    data_scaled = scaler.fit_transform(data_test)
    
    # Fit the PCA model and transform the data
    data_pca = pca.fit_transform(data_scaled)
    
    # Calculate the z-score for each point
    z_scores = np.abs(zscore(data_test)).max(axis=1)  # We only need the maximum z-score across features
    
    # Plot the transformed data, coloring by z-score and marking outliers
    plt.scatter(data_pca[:, 0], data_pca[:, 1], c=z_scores, marker='o', cmap='viridis')
    plt.scatter(data_pca[outliers_test, 0], data_pca[outliers_test, 1], c='red', marker='x')
    plt.title(f'PCA plot for test_no {test_no}')
    plt.xlabel('First principal component')
    plt.ylabel('Second principal component')
    plt.colorbar(label='Maximum z-score')
    plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Initialize a StandardScaler
scaler = StandardScaler()

# Initialize a KMeans model
kmeans = KMeans(n_clusters=3, random_state=42)

# Remove rows with NaN values
data = data.dropna()

# Try the clustering again
for test_no in data['test_no'].unique():

    # Select the data for this test_no
    data_test = data[data['test_no'] == test_no][features]
    
    # Scale the data
    data_scaled = scaler.fit_transform(data_test)
    
    # Fit the KMeans model
    kmeans.fit(data_scaled)
    
    # Calculate the distance from each point to its cluster center
    distances = kmeans.transform(data_scaled).min(axis=1)
    
    # Calculate the threshold (3 standard deviations from the mean distance)
    threshold = distances.mean() + 3*distances.std()
    
    # Mark outliers in the original dataframe
    data.loc[data['test_no'] == test_no, 'outlier'] = distances > threshold

# Display the first few rows of the updated data
data.head()


In [ ]:
# For each test_no
for test_no in data['test_no'].unique():

    # Select the data for this test_no
    data_test = data[data['test_no'] == test_no][features]
    outliers_test = data[data['test_no'] == test_no]['outlier']
    
    # Scale the data
    data_scaled = scaler.fit_transform(data_test)
    
    # Fit the PCA model and transform the data
    data_pca = pca.fit_transform(data_scaled)
    
    # Plot the transformed data, coloring by cluster and marking outliers
    plt.scatter(data_pca[:, 0], data_pca[:, 1], c=kmeans.predict(data_scaled), marker='o', cmap='viridis')
    plt.scatter(data_pca[outliers_test, 0], data_pca[outliers_test, 1], c='red', marker='x')
    plt.title(f'PCA plot for test_no {test_no}')
    plt.xlabel('First principal component')
    plt.ylabel('Second principal component')
    plt.show()


In [ ]:
# For each test_no
for test_no in data['test_no'].unique():

    # Select the data for this test_no
    data_test = data[data['test_no'] == test_no][features]
    
    # Scale the data
    data_scaled = scaler.fit_transform(data_test)
    
    # Fit the PCA model
    pca.fit(data_scaled)
    
    # Print the explained variance ratio of the principal components
    print(f'Explained variance ratio for test_no {test_no}: {pca.explained_variance_ratio_}')
    
    # Print the loadings of the principal components
    print(f'Loadings for test_no {test_no}:')
    loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=features)
    print(loadings)


In [ ]:
from sklearn.neighbors import NearestNeighbors

# Initialize a NearestNeighbors model
nn = NearestNeighbors(n_neighbors=4)  # The first nearest neighbor is the point itself

# For each test_no
for test_no in data['test_no'].unique():

    # Select the data for this test_no
    data_test = data[data['test_no'] == test_no][features]
    
    # Scale the data
    data_scaled = scaler.fit_transform(data_test)
    
    # Fit the NearestNeighbors model
    nn.fit(data_scaled)
    
    # Calculate the distance to the third nearest neighbor
    distances, _ = nn.kneighbors(data_scaled)
    distances = distances[:, -1]  # We only need the distance to the third nearest neighbor
    
    # Calculate the threshold (3 standard deviations from the mean distance)
    threshold = distances.mean() + 3*distances.std()
    
    # Mark outliers in the original dataframe
    data.loc[data['test_no'] == test_no, 'outlier'] = distances > threshold

# Display the first few rows of the updated data
data.head()


In [ ]:
# For each test_no
for test_no in data['test_no'].unique():

    # Select the data for this test_no
    data_test = data[data['test_no'] == test_no][features]
    outliers_test = data[data['test_no'] == test_no]['outlier']
    
    # Scale the data
    data_scaled = scaler.fit_transform(data_test)
    
    # Fit the PCA model and transform the data
    data_pca = pca.fit_transform(data_scaled)
    
    # Calculate the distance to the third nearest neighbor
    distances, _ = nn.kneighbors(data_scaled)
    distances = distances[:, -1]  # We only need the distance to the third nearest neighbor
    
    # Plot the transformed data, coloring by distance and marking outliers
    plt.scatter(data_pca[:, 0], data_pca[:, 1], c=distances, marker='o', cmap='viridis')
    plt.scatter(data_pca[outliers_test, 0], data_pca[outliers_test, 1], c='red', marker='x')
    plt.title(f'PCA plot for test_no {test_no}')
    plt.xlabel('First principal component')
    plt.ylabel('Second principal component')
    plt.colorbar(label='Distance to third nearest neighbor')
    plt.show()


In [ ]:
kw.writedf_xlsx({'sheet1': data_describe})